In [1]:
#%%% JSON content script
import pandas as pd
import json
from sqlalchemy import create_engine

# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dsql-maxaro-prod.database.windows.net%3B' \
                    'Database%3Dsqldb-maxaro-prod%3B' \
                    'UID%3Dbiadmin@maxaro.nl%3B' \
                    'Authentication%3DActiveDirectoryInteractive'

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT JsonContent, EmailContentType, [To]
         FROM Emails
         WHERE EmailContentType IN (
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.IncorrectDelivery.ServiceFormIncorrectDeliveryEmailContent',
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.Installation.ServiceFormInstallationEmailContent',
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.Defect.ServiceFormDefectEmailContent'
        );"""

# Get the email data as a Pandas DataFrame
json_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()


In [2]:
data = pd.DataFrame(json_data)

def is_valid_json(jstr):
    try:
        json.loads(jstr)
        return True
    except:
        return False

# Keep only valid JSON rows
valid_data = data[data['JsonContent'].apply(is_valid_json)].copy()

# Convert the 'JsonContent' string to a dictionary
valid_data['JsonContentDict'] = valid_data['JsonContent'].apply(json.loads)

# Extract the info from each dictionary
valid_data['ArticleNumber'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['ArticleNumber'] if 'Products' in x and x['Products'] else None)
valid_data['ProductIsAssembled'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['ProductIsAssembled'] if 'ProductIsAssembled' in x['Products'][0] else None)
valid_data['Comment'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['Comment'] if 'Comment' in x['Products'][0] else None)

# Extract 'OrderNumber' from each dictionary and remove any '#' from the start
valid_data['OrderNumber'] = valid_data['JsonContentDict'].apply(lambda x: x.get('OrderNumber', None)).str.lstrip('#')
for number in valid_data['OrderNumber']:
    number.upper()

# pattern for how an article number should look like
article_pattern = r'(A\d{7})'
valid_data['Type'] = 'Search'  # default all to Search and then change the ones that are Articles

# Classify 'ArticleNumber'
valid_data['ITEMNUMBER'] = valid_data['ArticleNumber'].str.extract(article_pattern)
valid_data.loc[valid_data['ArticleNumber'].str.match(article_pattern, na=False), 'Type'] = 'Article'
valid_data['searchData'] = valid_data.loc[valid_data['Type'] == 'Search', 'ArticleNumber']

# Classify 'OrderNumber' for both MNHQ and Others
mnhq_pattern = r'^MNHQ'
other_order_pattern = r'^(IMX|WMX|BMX)'
valid_data['MNHQ_Order'] = valid_data['OrderNumber'][valid_data['OrderNumber'].str.match(mnhq_pattern, na=False)]
valid_data['Other_Order'] = valid_data.loc[valid_data['OrderNumber'].str.match(other_order_pattern, na=False), 'OrderNumber']

In [3]:
json_data

,JsonContent,EmailContentType,To
0,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""e"",...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@egiedsimons.com
1,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Joh...",NetivityEcommerce.Core.Domain.Emails.Contents....,jhpvanyperen@gmail.com
2,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""K...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@krekelsbouwservice.nl
3,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Art...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@staling.nl
4,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Joo...",NetivityEcommerce.Core.Domain.Emails.Contents....,j.t.braun@gmail.com
...,...,...,...
752,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jo""...",NetivityEcommerce.Core.Domain.Emails.Contents....,Jpdewaal95@hotmail.com
753,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""E...",NetivityEcommerce.Core.Domain.Emails.Contents....,dionenemma@gmail.com
754,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""C. ...",NetivityEcommerce.Core.Domain.Emails.Contents....,cpbaaij@gmail.com
755,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Ale...",NetivityEcommerce.Core.Domain.Emails.Contents....,alex.noordzij@gmail.com


In [4]:
valid_data

,JsonContent,EmailContentType,To,JsonContentDict,ArticleNumber,ProductIsAssembled,Comment,OrderNumber,Type,ITEMNUMBER,searchData,MNHQ_Order,Other_Order
0,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""e"",...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@egiedsimons.com,"{'Gender': 'Men', 'FirstName': 'e', 'MiddleNam...",M40-0400-43080,False,Bij het uitpakken bleek de spiegel beschadigd ...,WMX782120,Search,NaN,M40-0400-43080,NaN,WMX782120
1,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Joh...",NetivityEcommerce.Core.Domain.Emails.Contents....,jhpvanyperen@gmail.com,"{'Gender': 'Men', 'FirstName': 'John', 'Middle...",A0006570,False,Muurprofiel van de wc-bak is totaal niet stevi...,PS511923,Article,A0006570,NaN,NaN,NaN
2,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""K...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@krekelsbouwservice.nl,"{'Gender': 'Women', 'FirstName': 'Krekels Bouw...",R0210-0909N,True,"Graag ontvangen wij een nieuwe rubberen strip,...",WMX722193,Search,NaN,R0210-0909N,NaN,WMX722193
3,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Art...",NetivityEcommerce.Core.Domain.Emails.Contents....,info@staling.nl,"{'Gender': 'Men', 'FirstName': 'Arthur', 'Midd...",501-060102,False,"1 pak wandtegels met breuk, waren al kapot op ...",MNHQSO0450968,Search,NaN,501-060102,MNHQSO0450968,NaN
4,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Joo...",NetivityEcommerce.Core.Domain.Emails.Contents....,j.t.braun@gmail.com,"{'Gender': 'Men', 'FirstName': 'Joost', 'Middl...",DR17-0617W,False,Schade aan handdoekradiator nog in de verpakki...,MNHQSO0465133,Search,NaN,DR17-0617W,MNHQSO0465133,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jo""...",NetivityEcommerce.Core.Domain.Emails.Contents....,Jpdewaal95@hotmail.com,"{'Gender': 'Men', 'FirstName': 'Jo', 'MiddleNa...",DS-001W,None,None,MNHQS00466858,Search,NaN,DS-001W,MNHQS00466858,NaN
753,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""E...",NetivityEcommerce.Core.Domain.Emails.Contents....,dionenemma@gmail.com,"{'Gender': 'Women', 'FirstName': 'Emma', 'Midd...",CDC108094310MB,None,None,MNHQSO0439221,Search,NaN,CDC108094310MB,MNHQSO0439221,NaN
754,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""C. ...",NetivityEcommerce.Core.Domain.Emails.Contents....,cpbaaij@gmail.com,"{'Gender': 'Men', 'FirstName': 'C. P.', 'Middl...",J01-2AH200869016,None,None,W01013,Search,NaN,J01-2AH200869016,NaN,NaN
755,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Ale...",NetivityEcommerce.Core.Domain.Emails.Contents....,alex.noordzij@gmail.com,"{'Gender': 'Men', 'FirstName': 'Alex', 'Middle...",None,None,None,0000,Search,NaN,None,NaN,NaN


In [5]:
#7
article_df = valid_data.loc[valid_data['Type'] == 'Article', ['ITEMNUMBER', 'MNHQ_Order', 'Other_Order', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType']]#.to_excel("C:\\Users\\bgraziadei\\OneDrive - Maxaro\\Documenten\\JsonContentarticle.xlsx", index=False)
search_df = valid_data.loc[valid_data['Type'] == 'Search', ['searchData', 'MNHQ_Order', 'Other_Order', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType']]#.to_excel("C:\\Users\\bgraziadei\\OneDrive - Maxaro\\Documenten\\JsonContentsearch.xlsx", index=False)

In [6]:
# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT T.SALESORDERNUMBER, T.CUSTOMERREQUISITIONNUMBER, T.MAXWEBINTEGRATIONSETCODEID, T.ITEMNUMBER
        FROM SalesOrderLineV2Staging T
        INNER JOIN SalesOrderHeaderV2Staging S ON S.SALESORDERNUMBER = T.SALESORDERNUMBER
        WHERE T.DATAAREAID = 'MNHQ';"""
# Get the order data as a Pandas DataFrame
order_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()

In [7]:
# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT T.Productnumber, T.ProductSearchname FROM [dbo].[EcoResProductV2Staging] T"""
# Get the order data as a Pandas DataFrame
product_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()

In [8]:
search_df = search_df.merge(product_data, how='inner', left_on='searchData', right_on='ProductSearchname')
search_df.rename(columns={'Productnumber': 'ITEMNUMBER'}, inplace=True)

In [9]:
article_df.dropna(subset=['MNHQ_Order', 'Other_Order'], how='all', inplace=True)
merge_key1 = article_df.merge(order_data, left_on = ['MNHQ_Order', 'ITEMNUMBER'], right_on = ['SALESORDERNUMBER', 'ITEMNUMBER'], how = 'left')         
merge_key2 = article_df.merge(order_data, left_on = ['Other_Order', 'ITEMNUMBER'], right_on = ['CUSTOMERREQUISITIONNUMBER', 'ITEMNUMBER'], how = 'left')
result_art = pd.concat([merge_key1, merge_key2], ignore_index=True)

In [10]:
search_df.dropna(subset=['MNHQ_Order', 'Other_Order'], how='all', inplace=True)
merge_key1 = search_df.merge(order_data, left_on = ['MNHQ_Order', 'ITEMNUMBER'], right_on = ['SALESORDERNUMBER', 'ITEMNUMBER'], how = 'left')         
merge_key2 = search_df.merge(order_data, left_on = ['Other_Order', 'ITEMNUMBER'], right_on = ['CUSTOMERREQUISITIONNUMBER', 'ITEMNUMBER'], how = 'left')
result_ser = pd.concat([merge_key1, merge_key2], ignore_index=True)

In [11]:
wanted_col = ['ITEMNUMBER', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType', 'SALESORDERNUMBER', 'CUSTOMERREQUISITIONNUMBER', 'MAXWEBINTEGRATIONSETCODEID']
result = pd.concat([result_art[wanted_col], result_ser[wanted_col]], ignore_index=True)

In [12]:
result.drop_duplicates(inplace=True)

In [13]:
result

,ITEMNUMBER,Comment,ProductIsAssembled,To,EmailContentType,SALESORDERNUMBER,CUSTOMERREQUISITIONNUMBER,MAXWEBINTEGRATIONSETCODEID
0,A0001733,The Sifon is leaking from the bottom. Picture ...,True,arun.balagi@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0446731,WMX764431,99.050.024
2,A0002964,Wederom hebben wij een tl-lamp kapot. \nDit is...,False,Estherwijdeven@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
3,A0008281,het product is enkel uitgepakt om te kijken na...,False,n.benkhattab@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0465038,BMX781150,BOB55-00004
5,A0009572,Wederom geleverd met beschadigde verpakking wa...,False,re.ko7510@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0466539,IMX782505,
7,A0007461,"Van het inbouwbad, ligbad Capri 190x90cm Acryl...",False,peter.douma@doumaprojectmangement.nl,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0461879,IMX778245,XZ03-1990XLSM
...,...,...,...,...,...,...,...,...
1084,A0008397,None,None,sandergebbink@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
1085,A0003045,None,None,kennyryken@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0478230,WMX793008,301-500101
1087,A0005614,None,None,vlad.smai.040796@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0478838,IMX793556,304-040301
1092,A0001679,None,None,Mulder.mh@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0463422,IMX779666,91101500


In [14]:
missing_values = result.isna().sum()

print(missing_values)

ITEMNUMBER                      0
Comment                       241
ProductIsAssembled            227
To                              0
EmailContentType                0
SALESORDERNUMBER              364
CUSTOMERREQUISITIONNUMBER     364
MAXWEBINTEGRATIONSETCODEID    364
dtype: int64


## load in and combine with previous file
_____________________________

In [15]:
prev = pd.read_excel(r"M:\ERP\Data\Setcodes\SetCodesMerged.xlsx")

In [17]:
comb = pd.concat([prev, result], ignore_index=True)

In [18]:
comb

,ITEMNUMBER,Comment,ProductIsAssembled,To,EmailContentType,SALESORDERNUMBER,CUSTOMERREQUISITIONNUMBER,MAXWEBINTEGRATIONSETCODEID
0,A0001953,"Bij het monteren van het toilet aan de frame, ...",0.0,asmaebay1@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0412397,WMX732994,TOI0003
1,A0010186,"Beste heer, mevrouw. \n\nGraag meld ik even he...",0.0,kerryhartman@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
2,A0012134,Schade aan de deur gebroken geleverd.,0.0,manu69@live.nl,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
3,A0008413,Na open maken verpakking zagen we dat de wasko...,0.0,jurgen.dehaas@softwareag.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0397483,IMX719495,Q120-0040
4,A0007531,Vrijdag 20-10-23 ook e-mail gestuurd aan info@...,1.0,m.marijnusse@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0405545,WMX726782,CDB214610320MB
...,...,...,...,...,...,...,...,...
2384,A0008397,None,None,sandergebbink@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
2385,A0003045,None,None,kennyryken@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0478230,WMX793008,301-500101
2386,A0005614,None,None,vlad.smai.040796@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0478838,IMX793556,304-040301
2387,A0001679,None,None,Mulder.mh@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0463422,IMX779666,91101500


In [19]:
comb.to_excel(r"M:\ERP\Data\Setcodes\SetCodesMerged.xlsx", index=False)